In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import metrics
from skimage.transform import resize
from PIL import Image
from collections import OrderedDict
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from dataset.RSNAtestDataset import *
import pickle
from torch.utils.data import DataLoader
# import seaborn as sns
# from sklearn.manifold import TSNE

/home/zhemin/anaconda3/envs/vqvae_env/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/zhemin/anaconda3/envs/vqvae_env/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Load TransBTS Model

In [5]:
from networks.HAND.Hybrid_dis_GR import TransMammo
_, model_bts = TransMammo(dataset='breast2', _conv_repr=True, _pe_type="learned")
checkpoint = torch.load('/home/zhemin/Datacenter_storage/zhemin/TransBTS/checkpoint_aux4_1/TransBTS2024-07-02/model_epoch_119.pth')
model_bts.load_state_dict(checkpoint['state_dict'])
# model_bts = load_ckpt('./checkpoint/TransBTS2024-04-10/model_epoch_last.pth', model_bts)
# model_bts.to(device)
model_bts.eval()

BTS(
  (linear_encoding): Linear(in_features=128, out_features=128, bias=True)
  (position_encoding): LearnedPositionalEncoding()
  (pe_dropout): Dropout(p=0.1, inplace=False)
  (transformer): TransformerModel(
    (net): IntermediateSequential(
      (0): Residual(
        (fn): PreNormDrop(
          (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (fn): SelfAttention(
            (qkv): Linear(in_features=128, out_features=384, bias=False)
            (attn_drop): Dropout(p=0.1, inplace=False)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (proj_drop): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (1): Residual(
        (fn): PreNorm(
          (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (fn): FeedForward(
            (net): Sequential(
              (0): Linear(in_features=128, out_features=256, bias=True)
              (1):

In [6]:
from sklearn.metrics import accuracy_score
from numpy import sqrt, argmax
from sklearn.metrics import auc, roc_curve

def get_fpr_tpr_auc(Y_label, Y_preds): 
    fpr, tpr, thresholds = roc_curve(Y_label, Y_preds)
    aucscore = auc(fpr, tpr)
    gmeans = sqrt(tpr * (1-fpr))
    ix = argmax(gmeans)
    #logger = logging.getLogger()
    print('Best Threshold=%f, G-Mean=%.3f, FPR=%.3f, TPR=%.3f, AUC=%.3f' % (thresholds[ix], gmeans[ix], fpr[ix], tpr[ix], aucscore))
    return thresholds[ix], fpr, tpr, aucscore

## Load Test Dataset

In [67]:
external_dataset = True
eval_set = 'overall'
dataset_name = 'breast2'
batch_size = 8
random_seed = 42
def build_test_set (dataset_name, batch_size):
    print("--------Building test set-------")
    print("--------" + eval_set + "-------")
    if external_dataset == False: # RSNA
        test_df_all = pd.read_csv('/home/zhemin/Datacenter_storage/zhemin/test_generation/4_test_all.csv') 
        
        normal_value = 'none'
        df_normal = test_df_all[test_df_all['noise'] == normal_value]

        if eval_set == 'overall':
            test_set = RSNAtestDataset(test_df_all)
        
        elif eval_set == 'normal':
            test_set = RSNAtestDataset(df_normal)

        elif eval_set == 'implant':
            # df_normal = df_normal.sample(n=100, random_state=random_seed)
            df_implant = test_df_all[test_df_all['noise'] == 'implant']
            df_implant = pd.concat([df_implant, df_normal])
            test_set = RSNAtestDataset(df_implant)

        elif eval_set == 'noise':
            # df_normal = df_normal.sample(n=100, random_state=random_seed)
            noise_value = ['salt_pepper', 'distort', 'gaussian']
            df_noise = test_df_all[test_df_all['noise'].isin(noise_value)]
            df_noise = pd.concat([df_noise, df_normal])
            test_set = RSNAtestDataset(df_noise)

        elif eval_set == 'poor_res':
            # df_normal = df_normal.sample(n=80, random_state=random_seed)
            df_poor = test_df_all[test_df_all['noise'] == 'poor_res']
            df_poor = pd.concat([df_poor, df_normal])
            test_set = RSNAtestDataset(df_poor)

        elif eval_set == 'natural':
            # df_normal = df_normal.sample(n=79, random_state=random_seed)
            df_natural = test_df_all[test_df_all['noise'] == 'natural']
            df_natural = pd.concat([df_natural, df_normal])
            test_set = RSNAtestDataset(df_natural)


    elif external_dataset == True: # Mayo
        test_df_all = pd.read_csv('/home/zhemin/Datacenter_storage/zhemin/test_generation/external/test_mayo_clean.csv')
        df_normal = test_df_all[test_df_all['noise']=='Null']
        if eval_set == 'overall':
            test_set = RSNAtestDataset(test_df_all)

        elif eval_set == 'normal':
            test_set = RSNAtestDataset(df_normal)
            
        elif eval_set == 'implant':
            #df_normal = df_normal.sample(n=64, random_state=random_seed)
            implant_value = ['Implant', 'Implant_rapture']
            df_implant = test_df_all[test_df_all['noise'].isin(implant_value)] 
            df_implant = pd.concat([df_implant, df_normal])
            test_set = RSNAtestDataset(df_implant)
            
        elif eval_set == 'biopsy_clip':
            #df_normal = df_normal.sample(n=37, random_state=random_seed)
            bc_value = ['biopsy_clip', 'biopsy_clip_&_Quality_Issue']
            df_bc = test_df_all[test_df_all['noise'].isin(bc_value)]
            df_bc = pd.concat([df_bc, df_normal])
            test_set = RSNAtestDataset(df_bc)
            
        elif eval_set == 'Quality_Issue':
            #df_normal = df_normal.sample(n=95, random_state=random_seed)
            df_qi = test_df_all[test_df_all['noise'] == 'Quality_Issue']
            df_qi_all = pd.concat([df_qi, df_normal])
            test_set = RSNAtestDataset(df_qi_all)

    test_dataloaders = DataLoader(test_set, batch_size = batch_size, shuffle = True, num_workers = 8)
    test_dataset_sizes = len(test_set)
    return test_dataloaders, test_dataset_sizes
        

In [68]:
test_loader, test_size = build_test_set('breast2', 16)
print(test_size)

--------Building test set-------
--------overall-------
220


### Use MSE + BCE loss for anomaly scores

In [69]:
# calculate anomaly score with MSE loss
from torchmetrics.image import StructuralSimilarityIndexMeasure
from torch.nn import functional as F

Targets = []
Paths = []
Ano_types = []

mse_l = []
bce_l = []
latent_sigmoids = []
ssim_loss = []
latent_test = []

with torch.no_grad():
    for idx, data in enumerate(test_loader):
        images, targets, anomaly_type, img_path = data
        for i in range(0, images.shape[0]):
            #print(i)
            print(anomaly_type[i])
            
            #print(images[i].unsqueeze(0).shape)
            Ano_types.append(anomaly_type[i])
            Targets.append(targets[i].item())
            Paths.append(img_path[i])
            
            #x1_1, x2_1, x3_1, x, intmd_x = model_bts.encode(images[i].unsqueeze(0))
            recon_x, z = model_bts(images[i].unsqueeze(0))

            # MSE
            mse_loss = F.mse_loss(recon_x, images[i].unsqueeze(0), reduction='sum')
            #print("MSE loss", mse_loss)
            mse_l.append(mse_loss.item())

            # BCE
            target_label = torch.zeros(1, 1)
            bce_loss = F.binary_cross_entropy(target_label, z)
            print(bce_loss)
            bce_l.append(bce_loss.item())

            # using sigmoid directly from latent space
            l_sigmoid = torch.sigmoid(z)
            print(l_sigmoid)
            latent_sigmoids.append(l_sigmoid.squeeze(0).item())

            # SSIM
            ssim = StructuralSimilarityIndexMeasure(data_range=1.0)
            ssim_loss.append(ssim(recon_x, images[i].unsqueeze(0)).item())



Null
tensor(0.0495)
tensor([[0.5001]])
Implant_rapture
tensor(100.0000)
tensor([[0.7311]])
Quality_Issue
tensor(0.1252)
tensor([[0.5003]])
Null
tensor(0.1255)
tensor([[0.5003]])
Null
tensor(0.0381)
tensor([[0.5001]])
Null
tensor(0.0926)
tensor([[0.5002]])
Null
tensor(0.2159)
tensor([[0.5005]])
Quality_Issue
tensor(0.0004)
tensor([[0.5000]])
biopsy_clip
tensor(1.6757)
tensor([[0.5042]])
biopsy_clip
tensor(0.0891)
tensor([[0.5002]])
Null
tensor(0.0216)
tensor([[0.5001]])
Null
tensor(0.0805)
tensor([[0.5002]])
Null
tensor(0.0584)
tensor([[0.5001]])
Null
tensor(0.1095)
tensor([[0.5003]])
Quality_Issue
tensor(5.9917)
tensor([[0.5150]])
Implant_rapture
tensor(0.0682)
tensor([[0.5002]])
biopsy_clip
tensor(0.0247)
tensor([[0.5001]])
Quality_Issue
tensor(0.0121)
tensor([[0.5000]])
Null
tensor(0.2147)
tensor([[0.5005]])
Null
tensor(0.0418)
tensor([[0.5001]])
Null
tensor(0.0322)
tensor([[0.5001]])
Null
tensor(0.2608)
tensor([[0.5007]])
Quality_Issue
tensor(0.4731)
tensor([[0.5012]])
Implant_raptu

In [70]:
df_ano = pd.DataFrame(columns=['anomaly_type', 'label', 'ssim', 'mse_loss',  'bce_loss', 'latent_sigmoid', 'path'])
# df_ano = pd.DataFrame(columns=['anomaly_type', 'label', 'ssim', 'mse_loss', 'path'])

df_ano['anomaly_type'] = Ano_types
df_ano['label'] = Targets
df_ano['mse_loss'] = mse_l
df_ano['bce_loss'] = bce_l
df_ano['latent_sigmoid'] = latent_sigmoids
df_ano['ssim'] = ssim_loss
df_ano['path'] = Paths

#df_ano['latent_dist'] = latent_dists

df_ano

,anomaly_type,label,ssim,mse_loss,bce_loss,latent_sigmoid,path
0,Null,0,0.866533,92.219528,0.049467,0.500124,/home/zhemin/Datacenter_storage/zhemin/test_ge...
1,Implant_rapture,1,0.861044,43.616062,99.999985,0.731059,/home/zhemin/Datacenter_storage/zhemin/test_ge...
2,Quality_Issue,1,0.892503,93.281845,0.125153,0.500313,/home/zhemin/Datacenter_storage/zhemin/test_ge...
3,Null,0,0.896674,111.215591,0.125538,0.500314,/home/zhemin/Datacenter_storage/zhemin/test_ge...
4,Null,0,0.866376,79.117416,0.038125,0.500095,/home/zhemin/Datacenter_storage/zhemin/test_ge...
...,...,...,...,...,...,...,...
215,Null,0,0.874153,118.279701,0.124616,0.500312,/home/zhemin/Datacenter_storage/zhemin/test_ge...
216,Implant,1,0.930869,173.268341,99.998894,0.731056,/home/zhemin/Datacenter_storage/zhemin/test_ge...
217,Null,0,0.864775,59.407330,0.260251,0.500651,/home/zhemin/Datacenter_storage/zhemin/test_ge...
218,Null,0,0.897412,42.142960,0.027137,0.500068,/home/zhemin/Datacenter_storage/zhemin/test_ge...


In [71]:
df_ano['path'].iloc[0]

'/home/zhemin/Datacenter_storage/zhemin/test_generation/external/test_png/Null_11655682_MLO.png'

### evaluate AUC and ACC

In [72]:
mse_l = df_ano['mse_loss']
bce_l = df_ano['bce_loss']
sigmoid_l = df_ano['latent_sigmoid']

In [73]:
Y_preds = []
ano_score1 = []
ano_score2 = []
#df_ano_pred = pd.DataFrame(columns=['anomaly_type', 'label', 'ssim', 'mse_loss', 'latent_dist_flip', 'pred_ano_score', 'path'])
df_ano_pred = pd.DataFrame(columns=['anomaly_type', 'label', 'ssim', 'mse_loss', 'bce_loss', 'latent_sigmoid', 'pred_ano_score', 'path'])

for s in mse_l:
    ano_score1.append((s-np.min(np.array(mse_l)))/(np.max(np.array(mse_l))-np.min(np.array(mse_l))))
    #Y_preds.append((s-np.min(np.array(mse_l)))/(np.max(np.array(mse_l))-np.min(np.array(mse_l))))
for s in sigmoid_l:
    ano_score2.append((s-np.min(np.array(sigmoid_l)))/(np.max(np.array(sigmoid_l))-np.min(np.array(sigmoid_l))))
for s1, s2 in zip(ano_score1, ano_score2):
    Y_preds.append(0.5*s1 + 0.5*s2)
df_ano_pred['anomaly_type'] = df_ano['anomaly_type']
df_ano_pred['label'] = df_ano['label']
df_ano_pred['mse_loss'] = mse_l
df_ano_pred['bce_loss'] = bce_l
df_ano_pred['latent_sigmoid'] = latent_sigmoids
df_ano_pred['pred_ano_score'] = Y_preds
df_ano_pred['ssim'] = df_ano['ssim']
df_ano_pred['path'] = df_ano['path']

In [74]:
df_ano_pred

,anomaly_type,label,ssim,mse_loss,bce_loss,latent_sigmoid,pred_ano_score,path
0,Null,0,0.866533,92.219528,0.049467,0.500124,0.006763,/home/zhemin/Datacenter_storage/zhemin/test_ge...
1,Implant_rapture,1,0.861044,43.616062,99.999985,0.731059,0.502463,/home/zhemin/Datacenter_storage/zhemin/test_ge...
2,Quality_Issue,1,0.892503,93.281845,0.125153,0.500313,0.007260,/home/zhemin/Datacenter_storage/zhemin/test_ge...
3,Null,0,0.896674,111.215591,0.125538,0.500314,0.008751,/home/zhemin/Datacenter_storage/zhemin/test_ge...
4,Null,0,0.866376,79.117416,0.038125,0.500095,0.005614,/home/zhemin/Datacenter_storage/zhemin/test_ge...
...,...,...,...,...,...,...,...,...
215,Null,0,0.874153,118.279701,0.124616,0.500312,0.009332,/home/zhemin/Datacenter_storage/zhemin/test_ge...
216,Implant,1,0.930869,173.268341,99.998894,0.731056,0.513220,/home/zhemin/Datacenter_storage/zhemin/test_ge...
217,Null,0,0.864775,59.407330,0.260251,0.500651,0.005180,/home/zhemin/Datacenter_storage/zhemin/test_ge...
218,Null,0,0.897412,42.142960,0.027137,0.500068,0.002486,/home/zhemin/Datacenter_storage/zhemin/test_ge...


In [75]:
thrsh_pred, fpr, tpr, aucscore = get_fpr_tpr_auc(df_ano_pred['label'], df_ano_pred['pred_ano_score'])
y_pred_thrsh = (Y_preds >= thrsh_pred).astype(int)
accuracy_s = accuracy_score(df_ano['label'], y_pred_thrsh)
print("accuracy:", accuracy_s)
print(np.mean(ssim_loss))
print(np.std(ssim_loss))

Best Threshold=0.010764, G-Mean=0.679, FPR=0.143, TPR=0.538, AUC=0.672
accuracy: 0.65
0.8714112841270186
0.05171517824533627


In [66]:
df_ano_pred.to_csv('eval_'+eval_set+'_dis_GR2.csv')